##### This Notebook is used to generate, Aggergate data from a ADW data source
##### from Silver layer
####  
##### And then load the change feed fromfact order output into a Lakehouse Gold Layer..
##### 
##### 
##### 
##### The particular dataset being used in this example is the ADWLT Sample SQL Source.

In [2]:

# Loading the necessary libraries
from datetime import datetime, timedelta
from dateutil import parser,relativedelta
import pyspark.sql.functions as F
from pyspark.sql.functions import year, month, dayofmonth, dayofweek, hour, to_date, col, quarter, explode, sequence, expr,current_timestamp,lit
from pyspark.sql.types import StructType, StructField, IntegerType, TimestampType, DoubleType, StringType, FloatType, ArrayType, LongType
from delta.tables import DeltaTable
from notebookutils import mssparkutils

StatementMeta(, 017de666-d222-4729-927f-c335da320f8d, 4, Finished, Available)

In [3]:
#df = spark.sql("DROP TABLE Gold.Fact_order_agg")


StatementMeta(, 017de666-d222-4729-927f-c335da320f8d, 5, Finished, Available)

In [4]:
%%sql
set spark.databricks.delta.properties.defaults.enableChangeDataFeed = true;

StatementMeta(, 017de666-d222-4729-927f-c335da320f8d, 6, Finished, Available)

<Spark SQL result set with 1 rows and 2 fields>

In [5]:
path_to_fact_order_change_feed="abfss://DataBash23@onelake.dfs.fabric.microsoft.com/SIlver.Lakehouse/Tables/fact_order"
path_to_gold_table="Tables/Fact_order_agg"

StatementMeta(, 017de666-d222-4729-927f-c335da320f8d, 7, Finished, Available)

In [6]:
default_date = datetime(2023,1,1)

if mssparkutils.fs.exists(path_to_gold_table):
 last_processed_timestamp =  spark.sql(f"select max(last_processed_timestamp) last_processed_timestamp from gold.Fact_order_agg").first().last_processed_timestamp
else:
  last_processed_timestamp =default_date  + timedelta(hours=0, minutes=0, seconds=0)





StatementMeta(, 017de666-d222-4729-927f-c335da320f8d, 8, Finished, Available)

In [7]:
print(last_processed_timestamp)

StatementMeta(, 017de666-d222-4729-927f-c335da320f8d, 9, Finished, Available)

2023-11-25 14:26:27.573000


In [8]:
%%sql
Select * from table_changes("silver.fact_order",2,10)

StatementMeta(, 017de666-d222-4729-927f-c335da320f8d, 10, Finished, Available)

<Spark SQL result set with 10 rows and 36 fields>

In [9]:

# Read the change feed data
fact_order_changes_df = spark.read.format("delta").option("readChangeFeed", "true").option("startingTimestamp", last_processed_timestamp).load(path_to_fact_order_change_feed)
# Rename '_commit_timestamp' and drop other reserved columns
fact_order_changes_df = (fact_order_changes_df
                          .withColumnRenamed("_commit_timestamp", "last_processed_timestamp")
                          .drop("_change_type", "_commit_version"))

fact_order_aggregated_df = fact_order_changes_df.groupBy("SalesOrderID", "SalesOrderDetailID").agg(
    F.max("last_processed_timestamp").alias("last_processed_timestamp"),
    F.sum("SubTotal").alias("TotalSubTotal"),
    F.sum("TaxAmt").alias("TotalTaxAmt"),
    F.sum("Freight").alias("TotalFreight"),
    F.sum("TotalDue").alias("TotalTotalDue"),
    
)


# Check if the Gold layer Delta table exists
if mssparkutils.fs.exists(path_to_gold_table):
    gold_delta_table = DeltaTable.forPath(spark, path_to_gold_table)
else:
    # Create the Gold layer Delta table if it doesn't exist
    fact_order_aggregated_df.write.format("delta").save(path_to_gold_table)
    gold_delta_table = DeltaTable.forPath(spark, path_to_gold_table)

# Define merge condition for the Gold layer (adjust as per your key columns)
merge_condition = (
    "sourceview.SalesOrderID = target.SalesOrderID AND "
    "sourceview.SalesOrderDetailID = target.SalesOrderDetailID"
)

# Execute merge into the Gold layer
gold_delta_table.alias("target").merge(
    fact_order_aggregated_df .alias("sourceview"), 
    merge_condition
).whenMatchedUpdateAll().whenNotMatchedInsertAll().execute()


StatementMeta(, 017de666-d222-4729-927f-c335da320f8d, 11, Finished, Available)

In [10]:

max_commit_timestamp = spark.sql(f"select max(last_processed_timestamp) last_processed_timestamp from gold.Fact_order_agg").first().last_processed_timestamp
print(max_commit_timestamp)

StatementMeta(, 017de666-d222-4729-927f-c335da320f8d, 12, Finished, Available)

2023-11-25 15:43:35.482000


In [11]:
df = spark.sql("SELECT * FROM Gold.Fact_order_agg LIMIT 1000")
display(df)

StatementMeta(, 017de666-d222-4729-927f-c335da320f8d, 13, Finished, Available)

SynapseWidget(Synapse.DataFrame, 0a482281-1e6f-4e87-bbe7-31b735510349)